In [1]:
import os
import json
import numpy as np
import pandas as pd

In [2]:
path_to_json_files = 'march18_vaccine_twitter/'

In [3]:
json_files = [pos_json for pos_json in os.listdir(path_to_json_files) if pos_json.endswith('.json')]

In [4]:
print(len(json_files))

2288


In [9]:
file_count = 0
tweet_id_text_list = []
tweet_ids = []

for file_name in json_files:
    with open(path_to_json_files + file_name) as json_data:
        data = json.load(json_data)
        
        data_df = pd.DataFrame.from_dict(data)
        
        # Filtering out non-English tweets and replies 
        data_df = data_df[data_df.lang == 'en']
        data_df = data_df[pd.isna(data_df.in_reply_to_screen_name)]
        data_df = data_df[pd.isna(data_df.in_reply_to_status_id)]
        data_df = data_df[pd.isna(data_df.in_reply_to_status_id_str)]
        data_df = data_df[pd.isna(data_df.in_reply_to_user_id)]
        data_df = data_df[pd.isna(data_df.in_reply_to_user_id_str)]
         
        # Iterating over every row of each json file
        for index, row in data_df.iterrows():
            tweet_id = row['id_str']
            
            # Checking if the row doesn't contain a re-tweet, and if the tweet has already been added to the dataset through a re-tweet 
            if pd.isna(row.retweeted_status) and tweet_id not in tweet_ids:                
                try:
                    tweet_text = row['extended_tweet']['full_text']
                except:
                    tweet_text = row['text']
                tweet_id_text_list.append([tweet_id, tweet_text])
                tweet_ids.append(tweet_id)
            elif not pd.isna(row.retweeted_status) and tweet_id not in tweet_ids:
                tweet_original_id = row['retweeted_status']['id_str'] 
                
                # Checking if the original tweet has already been added to the dataset
                if tweet_original_id not in tweet_ids:
                    try:
                        tweet_text = row['retweeted_status']['extended_tweet']['full_text']
                    except:
                        tweet_text = row['text']
                    tweet_id_text_list.append([tweet_original_id, tweet_text])
                    tweet_ids.append(tweet_original_id)
                    
    file_count += 1
    if file_count % 500 == 0:
        print("Finished: " + str(file_count) + " files")

Finished: 500 files
Finished: 1000 files
Finished: 1500 files
Finished: 2000 files


In [10]:
tweet_df = pd.DataFrame(tweet_id_text_list, columns=['id', 'Text'])

In [11]:
tweet_df

,id,Text
0,969228265901776897,Visit https://t.co/cd5fQhhm9i to order FREE re...
1,969151972166328321,March 3 is #WorldBDDay. Abt 8M #babies born wo...
2,969186912031379456,Wakefield's legacy lives on in #measles outbre...
3,969227439808438272,“If you look at the resources that are now ded...
4,969179891097919488,RT @sketchingsprw: *searching for Caleb gifs a...
5,965566172421935105,#giveaway #comp #win #FreebieFriday #winitwedn...
6,969228595855089665,First needle-free live vaccination device hera...
7,968778782931857410,A family in Nangarhar province #Afghanistan is...
8,969228697428377600,The Way This Anti-Vax Mom Reacted To Her 19-Ye...
9,969225786967445504,"Opinion: During his life, Sabin became a house..."
